## Setting up 

In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, LabelSet
import numpy as np

output_notebook()

Loading BokehJS ...

## Defining Variables

In [2]:
# long string variables
twentysix = [1,2,3,4,5,6,7,8,9,10,11,12,
             13,14,15,16,17,18,19,20,21,22,23,24,25,26]
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet = alphabet*2


# aesthetic variables
x_offset = -4.5    # letter annotation offset from circle edge
y_offset = -7    # letter annotation offset from circle edge

seg_color = 'gold'    # segment color
line_width = 3.0    # segment line width
seg_alpha = 0.6    # segment alpha

circ_color = 'gold'    # circle color
circ_size = 23    # circle size
circ_alpha = 0.4    # circle alpha
hov_color = 'gold'    # hover color for circles
hov_alpha = 0.8    # hover alpha for circles

font = 'Garamond'


# some points
x = np.array([64,69])
x = np.repeat(x,26)
y = twentysix*2

## Creating X and Y values for paths

In [3]:
# X values
xvals = [[64,69]]*26

In [4]:
# Y values
# rotor Gamma
yvals = [[1,6],[2,19],[3,15],[4,11],[5,1],[6,14],[7,21],[8,5],[9,18],
    [10,8],[11,13],[12,2],[13,20],[14,9],[15,25],[16,3],[17,23],
    [18,12],[19,17],[20,16],[21,26],[22,24],[23,22],[24,7],[25,10],
    [26,4]]

# Rotor Links - Rotor Gamma [elephant]

In [5]:
# establish links
links = {
    0:[31],     1:[44],      2:[40],     3:[36],     4:[26],     5:[39],      
    6:[46],     7:[30],      8:[43],     9:[33],    10:[38],    11:[27],   
   12:[45],    13:[34],     14:[50],    15:[28],    16:[48],    17:[37],    
   18:[42],    19:[41],     20:[51],    21:[49],    22:[47],    23:[32],    
   24:[35],    25:[29], 
       
   26:[4],    27:[11],    28:[15],    29:[25],    30:[7],    31:[0],
   32:[23],   33:[9],     34:[13],    35:[24],    36:[3],    37:[17],
   38:[10],   39:[5],     40:[2],     41:[19],    42:[18],   43:[8],
   44:[1],    45:[12],    46:[6],     47:[22],    48:[16],   49:[21],  
   50:[14],   51:[14]
}

## Create Graph and Add Features

In [6]:
# create graph
p = figure(width=400, height=800, tools="",
           toolbar_location=None,title='Hover over points')

# add letter labels
lsource = ColumnDataSource(data=dict(x = x, y = y, names = alphabet))
labels = LabelSet(x='x', y='y', text='names', level='annotation',
                  x_offset=x_offset, y_offset=y_offset, source=lsource,
                  render_mode='canvas', text_font=font, 
                  text_color='black', text_font_style='bold')
p.add_layout(labels)

# add hover glyphs
source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1':[]})
sr = p.segment(x0='x0',y0='y0',x1='x1', y1='y1', color=seg_color, 
               alpha=seg_alpha, line_width=line_width, source=source,)
cr = p.circle(x, y, color=circ_color, size=circ_size, alpha=circ_alpha, 
              hover_color=hov_color, hover_alpha=hov_alpha)

# add path glyphs
p.multi_line(xvals,yvals,color='gray',alpha=0.3)

# remove gridlines and axis labels
p.xgrid.visible = False
p.ygrid.visible = False
p.axis.visible = False

# JS for hover tool
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.get('data');
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.set('data', data);
""" % links

# Call JS and Add HoverTool
callback = CustomJS(args={'circle':cr.data_source, 
                          'segment':sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None,callback=callback,renderers=[cr]))

# Display Graph

In [7]:
show(p)

# Adding Outer Ring